In [11]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KFold (교차 검증을 사용하기 위해)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가 함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import  XGBRegressor

# 머신러닝 알고리즘 - 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 머신러닝 알고리즘 - 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 딥러닝 알고리즘 
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# 저장
import pickle

# 그래프 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'   # 윈도우용
plt.rcParams['font.family'] = 'AppleGothic'   # 맥용
plt.rcParams['font.size'] = 16                  # 폰트 크기
plt.rcParams['figure.figsize'] = 10,8          # 그래프 크기
plt.rcParams['axes.unicode_minus'] = False     # - 기호 깨짐 방지


# 경고 메시지가 안나오게 하기
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# 랜덤 시드값을 설정한다.
# 텐서플로에서 최초의 가중치와 바이어스를 랜덤하게 설정해서 시작한다.
# 랜덤 시드를 고정하면 예제를 실행할 때마다 같은 패턴의 랜덤으로 설정할수 있다.
# 앞으로 테스트해보는 예제에서 모든 조건을 동일하게 맞추고 테스트를 해야 정확한 테스트를 할 수 있기 때문에
# 가중치와 바이어스의 최초값을 항상 같은 값을 갖게 하도록 랜덤시드를 설정했다.
np.random.seed(3)
tf.random.set_seed(3)

In [3]:
# 데이터를 읽어온다.
df1 = pd.read_csv('../dataset/ThoraricSurgery.csv')
df1.head()

,293,1,3.8,2.8,0,0.1,0.2,0.3,0.4,0.5,12,0.6,0.7,0.8,1.1,0.9,62,0.10
0,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
1,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
2,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
3,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
4,18,2,2.96,1.67,0,0,0,0,0,0,12,0,0,0,1,0,61,0


In [4]:
# 입력과 결과로 나눈다.
x = df1.drop('0.10', axis=1)
y = df1['0.10']

In [5]:
# 딥러닝 구조를 설정
# 은닉층, 노드가 많아지면 계산이 많아지 때문에 보다 정확해질수 있다.
# 그러나 과적합의 우려가 있고 시간이 오래 걸린다.

# 딥러닝의 구조를 짜고 층을 설정하는 부분
model = Sequential()

# 첫번째 은닉층은 입력층도 같이 생성
# 첫번째 숫자 : 층 내부의 노드의 개수
# input_dim : 입력층의 노드 개수, 입력데이터의 컬럼의 수와 동일하게 설정
# activation : 활성화 함수, 가중치 손실을 예방하기 위해 'relu'로 설정
model.add(Dense(30, input_dim = 17, activation = 'relu'))

# 출력층 (예측하는 결과의 개수에 맞춰준다.)
# activation : 활성화 함수, 출력 결과에 따라 그에 맞는 활성화 함수를 사용한다.
model.add(Dense(1, activation = 'sigmoid'))

In [6]:
# 딥러닝 구조를 컴파일한다.
# 위에서 설정한 각 층의 정보를 토대로 딥러닝 모델을 생성한다.
# loss : 손실함수, 예측결과와 진짜 결과의 오차 정도를 계산하는 함수
# metrics : 손실함수를 통해 구한 오차 정도를 평가할 지표를 설정
# optimizer : 손실정도에 따라 가중치를 수정하는 경사하강법 설정

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['accuracy'])

In [20]:
# 학습시작
# epochs : 학습과 오차수정 (역전파) 작업을 얼마나 시도할 것인지
# 최적의 값으로 수정했는데도 계속 작업을 해나가면 오히려 성능이 떨어지는 경우가 있을수 있다.
# 적당한 선에서 끊어줘야한다.

# batch_size : 한번에 몇개의 row를 추출해서 학습시킬것인지? 
# 메모리가 허락하는한 최대의 숫자를 잡아주면 되지만 적게 주더라도 크게 차이가 나지는 않는다.
model.fit(x,y, epochs=100, batch_size=10)

Epoch 1/100
47/47 [==============================] - 0s 627us/step - loss: 0.1132 - accuracy: 0.8721
Epoch 2/100
47/47 [==============================] - 0s 635us/step - loss: 0.1105 - accuracy: 0.8678
Epoch 3/100
47/47 [==============================] - 0s 682us/step - loss: 0.1088 - accuracy: 0.8699
Epoch 4/100
47/47 [==============================] - 0s 631us/step - loss: 0.1140 - accuracy: 0.8763
Epoch 5/100
47/47 [==============================] - 0s 606us/step - loss: 0.1125 - accuracy: 0.8699
Epoch 6/100
47/47 [==============================] - 0s 615us/step - loss: 0.1125 - accuracy: 0.8742
Epoch 7/100
47/47 [==============================] - 0s 823us/step - loss: 0.1124 - accuracy: 0.8742
Epoch 8/100
47/47 [==============================] - 0s 829us/step - loss: 0.1093 - accuracy: 0.8721
Epoch 9/100
47/47 [==============================] - 0s 689us/step - loss: 0.1136 - accuracy: 0.8721
Epoch 10/100
47/47 [==============================] - 0s 652us/step - loss: 0.1104 - accura

47/47 [==============================] - 0s 616us/step - loss: 0.1105 - accuracy: 0.8635
Epoch 82/100
47/47 [==============================] - 0s 576us/step - loss: 0.1121 - accuracy: 0.8763
Epoch 83/100
47/47 [==============================] - 0s 547us/step - loss: 0.1154 - accuracy: 0.8678
Epoch 84/100
47/47 [==============================] - 0s 544us/step - loss: 0.1053 - accuracy: 0.8699
Epoch 85/100
47/47 [==============================] - 0s 552us/step - loss: 0.1052 - accuracy: 0.8742
Epoch 86/100
47/47 [==============================] - 0s 590us/step - loss: 0.1081 - accuracy: 0.8763
Epoch 87/100
47/47 [==============================] - 0s 543us/step - loss: 0.1047 - accuracy: 0.8699
Epoch 88/100
47/47 [==============================] - 0s 556us/step - loss: 0.1072 - accuracy: 0.8721
Epoch 89/100
47/47 [==============================] - 0s 561us/step - loss: 0.1078 - accuracy: 0.8742
Epoch 90/100
47/47 [==============================] - 0s 569us/step - loss: 0.1097 - accuracy: 

In [8]:
r1 = model.evaluate(x,y)[0]
r2 = model.evaluate(x,y)[1]

print(f'손실률 : {r1}')
print(f'정확도 : {r2}')

15/15 [==============================] - 0s 598us/step - loss: 0.1160 - accuracy: 0.8614
손실률 : 0.11598879098892212
정확도 : 0.8614072203636169


In [17]:
# 예측 결과를 보여줌 (_classes는 이제 사용하지 않음)
# result = model.predict_classes(x)

# 이진 분류인 경우
result = (model.predict(x) > 0.5).astype("int32")


In [18]:
# 정확도 확인
r1 = accuracy_score(y, result)
r1

0.8614072494669509